# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `grade_local.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [151]:
import pandas
import re
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11
0,2,1675 m^3,1.4804,computer science,"Technology And Information Management, Statist...",1.2781,2160,0.4820,0.2346,-571,RAT,1.0228,188
1,2,786 m^3,1.659,Computer Science,Natural Language Processing,0.5615,611,0.6517,0.0354,1773,?,0.3213,1258
2,1,133 m^3,-0.142,Computer Engineering,"Games and Playable Media, Computer Science",0.7352,1255,0.0109,0.2978,659,pig,1.6093,1259
3,2,2086 m^3,0.5385,Computer Science,Natural Language Processing,0.3008,-546,-0.4055,1.298,-779,DOG,-0.3082,-772
4,2,598 m^3,0.2668,Computer Science,Natural Language Processing,0.5279,-621,-0.3749,-0.978,1059,rat,0.0595,-781
...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,1,338 m^3,0.2614,Electrical Engineering,Games and Playable Media,0.1847,918,0.1282,1.4305,577,pig,0.2305,1377
964,1,2258 m^3,0.6303,Computer Engineering,"Statistics, Technology and Information Managem...",0.9535,-706,0.9705,-1.342,702,pig,1.4795,919
965,0,-18 m^3,0.4486,Games and Playable Media,Computational Media,0.4255,1064,-0.5959,0.9768,316,"guinea pig, rabbit",-0.4767,142
966,0,1138 m^3,0.9675,Computational Media,"Bioinformatics, Computational Media, Bioengine...",-0.1811,1213,0.6575,1.6594,1837,"hamster, guinea pig",0.9221,426


Don't forget to checkout the column information.

In [152]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   968 non-null    int64  
 1   col_00  958 non-null    object 
 2   col_01  961 non-null    object 
 3   col_02  967 non-null    object 
 4   col_03  965 non-null    object 
 5   col_04  956 non-null    object 
 6   col_05  958 non-null    object 
 7   col_06  962 non-null    float64
 8   col_07  964 non-null    object 
 9   col_08  961 non-null    object 
 10  col_09  958 non-null    object 
 11  col_10  961 non-null    object 
 12  col_11  965 non-null    object 
dtypes: float64(1), int64(1), object(11)
memory usage: 98.4+ KB


And any numeric information.

In [153]:
unique_data.describe()

,label,col_06
count,968.000000,962.000000
mean,1.995868,0.280398
std,1.423682,0.772522
min,0.000000,-2.020900
25%,1.000000,-0.242350
50%,2.000000,0.238600
75%,3.000000,0.795075
max,4.000000,2.793200


<h4 style="color: darkorange";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [155]:
test_data = pandas.DataFrame({"int_col": ["1", "1 (junk)"], "float_col": ["1.0", "1.0 (junk)"]})
test_data

,int_col,float_col
0,1,1.0
1,1 (junk),1.0 (junk)


In [ ]:
def clean_data(frame):
    # Drop rows with any NaN values
    frame = frame.dropna()

    def clean_column(data):
        cleaned_data = []
        for item in data:
            # Using regular expression to find numbers
            found_numbers = re.findall(r'[+-]?\d+(?:\.\d+)?', item)
            if found_numbers:
                number = found_numbers[0]
                # Try to convert to int or float
                if "." in number:
                    cleaned_data.append(float(number))
                else:
                    cleaned_data.append(int(number))
            else:
                # If no number found, append NaN
                cleaned_data.append(0)
        return cleaned_data
    
    # Apply cleaning to each column
    for col in frame.columns:
        frame[col] = clean_column(frame[col])

    return frame


unique_data = clean_data(unique_data)
unique_data

Now we should also be able to view all the numeric columns.

In [120]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 894 entries, 0 to 967
Data columns (total 57 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   label                                      894 non-null    float64
 1   col_00                                     894 non-null    object 
 2   col_01                                     894 non-null    object 
 3   col_04                                     894 non-null    object 
 4   col_05                                     894 non-null    object 
 5   col_06                                     894 non-null    float64
 6   col_07                                     894 non-null    object 
 7   col_08                                     894 non-null    object 
 8   col_10                                     894 non-null    object 
 9   col_11                                     894 non-null    object 
 10  pig                            

<h4 style="color: darkorange";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [159]:
def create_classifiers():
    # Create a Logistic Regression classifier
    lr = LogisticRegression()
    
    # Create a K-Nearest Neighbors classifier
    knn = KNeighborsClassifier()
    
    # Create a Decision Tree classifier
    dt = DecisionTreeClassifier()

    return [lr, knn, dt]


my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(), KNeighborsClassifier(), DecisionTreeClassifier()]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [ ]:
def cross_fold_validation(classifier, frame, folds):
    # Make a copy of the frame
    frame_copy = frame.copy()
    
    # Split frame into features (X) and labels (y)
    X = frame_copy.drop('label', axis=1)
    y = frame_copy['label']
    
    # Perform k-fold cross-validation
    accuracy_scores = cross_val_score(classifier, X, y, cv=folds)
    
    return accuracy_scores.tolist()

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

<h4 style="color: darkorange";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [ ]:
def significance_test(a_values, b_values, p_value):
    # Run independent two-sample t-test
    t_stat, p_val = stats.ttest_ind(a_values, b_values)
    
    # Check if the returned p-value is less than the threshold
    if p_val < p_value:
        return True
    else:
        return False


for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

<h4 style="color: darkorange";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.